# Hello Quantum World

Today, you’ll learn how to run code on a real IBM quantum computer. By the end,
you’ll be able to create a simple quantum program that links two quantum bits
(qubits) together. This connection between qubits is a key part of how quantum
computers work. Let’s dive in and see quantum computing in action!

### First time using a jupyter notebook? 

Press the play ▶ button next to the code cells below to run code in each cell. Follow the git below to see how to run the code cells in this notebook:

<img src="./images/how-to-run-a-jupyter-code-cell.gif" alt="how to run a code cell" width="900"/>

## Install the proper tools

In [ ]:
pip install qiskit-ibm-runtime qiskit[visualization]

- This command installs special tools on your  Grader Than Workspace that let you use IBM’s quantum computers and create visuals to help understand quantum code.
- By running this code on your Grader Than Workspace, you’ll be ready to write, test, and see quantum code in action, even if it’s your first time with quantum computing.

## Connect to the IBM Quantum Cloud

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService
import random

tokens = []

token = random.choice(tokens)
 
service = QiskitRuntimeService(channel="ibm_quantum", token=token)

- This code connects you to IBM’s quantum computer system using a secure token. 
- By creating this connection, you can send and run your quantum code on IBM’s quantum computers directly from this notebook.

## Create the quantum circuit

In [ ]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import EstimatorV2 as Estimator

# Set up a circuit with two qubits
qc = QuantumCircuit(2)

# Add a Hadamard gate to the first qubit
qc.h(0)

# Link the first qubit to the second using a controlled-X gate
qc.cx(0, 1)

# Show a visual drawing of the circuit
qc.draw("mpl")

- This code creates a **quantum circuit** with two qubits, which is like designing a path for our quantum information to follow.
- The **Hadamard gate** on the first qubit puts it into a state where it can
  interact with the second qubit, and the **controlled-X gate** links the two qubits,
  creating an "entangled" state—a key concept in quantum computing.
- Finally, `qc.draw("mpl")` displays a visual of the circuit, making it easy to see each step of our quantum circuit.

<details>
  <summary>🙋🏻‍♀️ What is a gate?</summary>
  
  A gate in quantum computing is an operation that changes the state of a qubit,
  similar to how a logic gate works in classical computing. Gates manipulate
  qubits in specific ways to create the desired outcomes in a quantum circuit. 
</details>

## Setup what we'd like to observe


In [ ]:
# Define six different measurements (observables) for the circuit
observables_labels = ["IZ", "IX", "ZI", "XI", "ZZ", "XX"]
observables = [SparsePauliOp(label) for label in observables_labels]

- This code sets up **six measurements** (called observables) to help us get information from the quantum circuit.
- Each measurement type is created based on different letter codes, like "IZ" and "XX," which tell the circuit what kind of data to collect.

<details>
  <summary>🤔 Learn more about these observables</summary>
  
Here’s a simple breakdown of each observable:

- **IZ**: Measures the Z (phase) property of the second qubit while ignoring the first qubit.
- **IX**: Measures the X (flip) property of the second qubit while ignoring the first qubit.
- **ZI**: Measures the Z (phase) property of the first qubit while ignoring the second qubit.
- **XI**: Measures the X (flip) property of the first qubit while ignoring the second qubit.
- **ZZ**: Measures the Z (phase) property of both qubits together, showing how their phases relate.
- **XX**: Measures the X (flip) property of both qubits together, showing how their flips relate.

Each observable helps us understand different aspects of the qubits' states.
</details>

<details>
  <summary>🙋🏽‍♀️ Why do we need observables?</summary>
  
Observables are essential because they allow us to measure and gather
information from a quantum circuit. In quantum computing, we can’t directly
observe qubits without collapsing their state, so observables give us a
structured way to extract specific data (like phase or flip properties) that
reveals the outcomes of our quantum operations. By setting up different
observables, we can capture various aspects of the qubits' behavior and
relationships, helping us understand the results of our quantum calculations. 

</details>

## Optimize the circuit for the quantum computer

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService

# Select the least busy available quantum computer (not a simulator)
backend = service.least_busy(simulator=False, operational=True)

# Optimize the circuit for the selected quantum computer
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(qc)

# Display a visual of the optimized circuit, showing only active qubits
isa_circuit.draw('mpl', idle_wires=False)

- This code picks the **least busy quantum computer** available to run the
  circuit faster. 
- It **prepares and optimizes the circuit** to work best on the selected quantum
  computer.
- Finally, it **shows a picture of the optimized circuit**, displaying only the
  active qubits. 

<details>
  <summary>🙋🏿‍♂️ Why do we need to prepares and optimizes the circuit?</summary>
  
Preparing and optimizing the circuit is essential because quantum computers have unique hardware constraints—such as specific qubit connections and noise levels—that can impact how well a circuit performs. Optimization:

1. **Increases Efficiency**: By adjusting the circuit to the hardware’s layout, it reduces the number of operations, making the circuit faster and more reliable.
2. **Improves Accuracy**: Minimizing unnecessary operations reduces errors, which helps produce clearer and more accurate results from the quantum computer.

In short, optimization helps make the most out of the quantum computer’s resources, ensuring better performance and accuracy in the results.

</details>

## Run the circuit

In [ ]:
# Set up the Estimator to run the circuit with our settings
estimator = Estimator(mode=backend)
estimator.options.resilience_level = 1
estimator.options.default_shots = 1000

# Adjust each measurement to match the layout of our circuit
mapped_observables = [
    observable.apply_layout(isa_circuit.layout) for observable in observables
]

# Run the circuit on the quantum computer with our measurements
job = estimator.run([(isa_circuit, mapped_observables)])

# Print a job ID so we can check the results later
print(f"Job ID: {job.job_id()}")

- This code prepares an **Estimator** to run the circuit we created and optimized earlier 1000 times, with basic error checking, and aligns the measurements to match the circuit layout.
- It then **sends the circuit to a quantum computer to run** and creates a job ID, allowing you to check your results later.

<details>
  <summary>🤔 The roll of the Estimator</summary>
  
Quantum computers are affected by noise, which means that the results of a
single run can be unreliable. By running the circuit many times (like 1000), we
can gather more data and calculate the average outcome, which helps reduce the
impact of errors and gives us a more accurate result.

An Estimator in quantum computing is a tool that calculates the expected
(average) result of measurements from the quantum circuit. It takes multiple
measurements (from each run) and processes them to provide a single, reliable
estimate of the outcome based on the circuit and observables we set up. 

</details>

## Wait for the Circuit to finish running on the Quantum Computer

In [ ]:
# Wait for the circuit run to be completed. Get the overall result from running the circuit
job_result = job.result()

# Extract the specific results for our six measurements
pub_result = job.result()[0]

- Once the job is finished, `job.result()` retrieves the **overall result** from running the circuit, including details about the entire submission and some extra information.
- `pub_result` pulls out the **specific results for our six measurements (observables)**, showing the data collected for each one in the circuit run.

<details>
  <summary>😫 Why do we have to wait so long for the circuit to finish?</summary>
  
We have to wait because there are only a limited number of quantum computers in
the world, and these powerful machines are shared by researchers and students
globally. Quantum computing is at the **cutting edge of technology and
innovation**; by running circuits on these rare and complex machines, you’re
part of an incredible moment in science. Every calculation you make is part of a
worldwide journey to unlock new ways of solving problems that classical
computers can’t handle, putting you on the frontier of the future of computing! 

</details>

## Graph the results

In [ ]:
from matplotlib import pyplot as plt

values = pub_result.data.evs

errors = pub_result.data.stds

# Display the results in a graph
plt.plot(observables_labels, values, '-o')
plt.xlabel('Observables')
plt.ylabel('Values')
plt.show()

- This code extracts the **measurement results** (`values`) and **errors** (`errors`) from the job results.
- It then **plots a graph** showing the values of each observable measurement, using the observable labels on the x-axis.

### What does this graph mean?

The high values for **"ZZ"** and **"XX"** in the graph suggest that the two qubits are **entangled**. Here’s why:

1. **Entanglement** means the qubits are linked in a way that their states are correlated. When we entangle qubits, measuring one qubit gives us information about the other, even if they’re physically separated. In our circuit, we entangled the qubits using a Hadamard gate on the first qubit followed by a controlled-X (CX) gate, which creates a specific kind of correlation between them.

2. **"ZZ" and "XX" Observables**: The "ZZ" observable measures the correlation in the phase states of both qubits, while "XX" measures the correlation in their flip states. Since both observables show high values close to 1, this means that:
   - When we measure one qubit in the **Z-basis** (related to "ZZ"), we get a predictable outcome for the other.
   - Similarly, measuring in the **X-basis** (related to "XX") also gives consistent, correlated results.
   
   These strong correlations in both Z and X measurements indicate that the qubits are entangled because the outcome of measuring one qubit directly influences the expected result of the other, regardless of the basis (phase or flip).

In summary, the high "ZZ" and "XX" values are evidence of entanglement, as they
show the qubits' states are connected in a way that one qubit’s measurement
affects the other.

<details>
  <summary>🤷🏼 What is a phase state and why it's important? (ZZ)</summary>
  
The **phase state** of a qubit describes its "angle" around the Z-axis. It
doesn’t change whether the qubit is 0 or 1, but it affects how qubits work
together, especially when they’re entangled. Phase is important because it helps
control the results of quantum operations. 

</details>

<details>
  <summary>🤷🏾 What is a flip state? (XX)</summary>
  
The **flip state** of a qubit refers to whether the qubit is in the 0 or 1 position. In quantum terms, the X gate (also known as the "flip" or "NOT" gate) switches a qubit from 0 to 1 or from 1 to 0, just like flipping a switch.

The **Controlled-X (CX) gate** uses this flip operation in a special way. It links two qubits: a **control qubit** and a **target qubit**. Here’s how it works:

1. If the **control qubit** is in state 1, the CX gate flips the **target qubit** (changing it from 0 to 1 or 1 to 0).
2. If the **control qubit** is in state 0, the target qubit stays the same.

This creates a link between the two qubits, which is key to creating **entanglement**. When qubits are entangled through a CX gate, their states become connected, meaning the state of one qubit affects the other. This connection is the foundation of many quantum algorithms.

</details>